<a href="https://colab.research.google.com/github/MnCSSJ4x/Global-Warming-Detection/blob/main/Time_Series_Analysis_of_Land_Temperature_using_MODIS_DRAFT1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as nn 
from tensorflow import keras

In [ ]:
print(tf.__version__)

2.11.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
table = pd.read_csv('drive/MyDrive/modisMYD_land_temp.csv')

In [ ]:
table 

,system:index,date,temperature,.geo
0,2010_01_15,2010-01-15,279.431458,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,2010_01_16,2010-01-16,294.972568,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,2010_01_17,2010-01-17,NaN,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,2010_01_18,2010-01-18,294.247037,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,2010_01_19,2010-01-19,292.215785,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...
4711,2022_12_26,2022-12-26,NaN,"{""type"":""MultiPoint"",""coordinates"":[]}"
4712,2022_12_27,2022-12-27,292.369203,"{""type"":""MultiPoint"",""coordinates"":[]}"
4713,2022_12_28,2022-12-28,294.076575,"{""type"":""MultiPoint"",""coordinates"":[]}"
4714,2022_12_29,2022-12-29,295.502888,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [ ]:
table.dropna(inplace=True)
table.reset_index(inplace=True)

In [ ]:
table.drop(labels=['index'],inplace=True,axis="columns")

In [ ]:
table 

,system:index,date,temperature,.geo
0,2010_01_15,2010-01-15,279.431458,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,2010_01_16,2010-01-16,294.972568,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,2010_01_18,2010-01-18,294.247037,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,2010_01_19,2010-01-19,292.215785,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,2010_01_20,2010-01-20,291.704135,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...
3645,2022_12_25,2022-12-25,290.720754,"{""type"":""MultiPoint"",""coordinates"":[]}"
3646,2022_12_27,2022-12-27,292.369203,"{""type"":""MultiPoint"",""coordinates"":[]}"
3647,2022_12_28,2022-12-28,294.076575,"{""type"":""MultiPoint"",""coordinates"":[]}"
3648,2022_12_29,2022-12-29,295.502888,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [ ]:
def get_time_series_data(sequence,n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
 # find the end of this pattern
    end_ix = i + n_steps
 # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
 # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)

  return np.array(X), np.array(y)


In [ ]:
n_steps = 3 

In [ ]:
X,y = get_time_series_data(table['temperature'].to_numpy(),3)

In [ ]:
X

array([[279.43145795, 294.97256785, 294.24703745],
       [294.97256785, 294.24703745, 292.21578471],
       [294.24703745, 292.21578471, 291.70413544],
       ...,
       [292.20726524, 290.72075436, 292.36920278],
       [290.72075436, 292.36920278, 294.0765753 ],
       [292.36920278, 294.0765753 , 295.50288791]])

In [ ]:
y

array([292.21578471, 291.70413544, 293.30386602, ..., 294.0765753 ,
       295.50288791, 293.76281921])

In [ ]:
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [ ]:
X[0]

array([[279.43145795],
       [294.97256785],
       [294.24703745]])

In [ ]:
X.shape

(3647, 3, 1)

In [ ]:
# define model
model = Sequential()
model.add(nn.LSTM(100, activation='relu', input_shape=(n_steps, n_features)))
model.add(nn.Dense(1))
opt = keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer=opt, loss='mae')

In [ ]:
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
114/114 - 3s - loss: 300.4587 - 3s/epoch - 24ms/step
Epoch 2/200
114/114 - 2s - loss: 192.4201 - 2s/epoch - 19ms/step
Epoch 3/200
114/114 - 3s - loss: 28.0583 - 3s/epoch - 25ms/step
Epoch 4/200
114/114 - 1s - loss: 3.0541 - 944ms/epoch - 8ms/step
Epoch 5/200
114/114 - 1s - loss: 3.0338 - 1s/epoch - 12ms/step
Epoch 6/200
114/114 - 2s - loss: 3.0489 - 2s/epoch - 17ms/step
Epoch 7/200
114/114 - 1s - loss: 3.0313 - 1s/epoch - 12ms/step
Epoch 8/200
114/114 - 1s - loss: 3.0392 - 1s/epoch - 9ms/step
Epoch 9/200
114/114 - 1s - loss: 3.0310 - 1s/epoch - 9ms/step
Epoch 10/200
114/114 - 2s - loss: 3.0141 - 2s/epoch - 13ms/step
Epoch 11/200
114/114 - 1s - loss: 3.0057 - 1s/epoch - 9ms/step
Epoch 12/200
114/114 - 2s - loss: 3.0287 - 2s/epoch - 16ms/step
Epoch 13/200
114/114 - 1s - loss: 2.9862 - 1s/epoch - 12ms/step
Epoch 14/200
114/114 - 1s - loss: 2.9973 - 829ms/epoch - 7ms/step
Epoch 15/200
114/114 - 1s - loss: 3.0372 - 792ms/epoch - 7ms/step
Epoch 16/200
114/114 - 1s - loss: 2.9998 